In [1]:
pip install scipy --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tenacity --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tiktoken --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install termcolor --quiet

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install requests --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  
import requests

GPT_MODEL = "gpt-3.5-turbo"
#GPT_MODEL = "gpt-4o"
client = OpenAI()

#First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"): #GPT가 함수를 호출하는 경우
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"): #GPT 일반 응답
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function": #함수결과 반환 메시지
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

#Let's create some function specifications to interface with a weather API. 
#We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
]

#Define the OpenWeatherMap API key and endpoint
#API_KEY = "your_openweathermap_api_key"
API_KEY = "api-key" #Please create your own API key and use it
CURRENT_WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"
#FORECAST_WEATHER_URL = "https://api.openweathermap.org/data/2.5/forecast"


#Now let's implement the functions that will actually fetch the real-time data from the OpenWeatherMap API.
#Function to get current weather
def get_current_weather(location, format="celsius"):
    units = "metric" if format == "celsius" else "imperial"
    params = {
        "q": location,
        "appid": API_KEY,
        "units": units
    }
    response = requests.get(CURRENT_WEATHER_URL, params=params)
    
    if response.status_code == 200:
        return response.json()  # The data fetched from OpenWeatherMap
    else:
        return {"error": "Failed to fetch current weather data"}



In [8]:
#Steps to invoke a function call using Chat Completions API:
#Step 1: Prompt the model with content that may result in model selecting a tool to use. 
    #The description of the tools such as a function names and signature is defined in the 'Tools' list and passed to the model in API call. 
    #If selected, the function name and parameters are included in the response.
#Step 2: Check programmatically if model wanted to call a function. If true, proceed to step 3.
#Step 3: Extract the function name and parameters from response, call the function with parameters. Append the result to messages.
#Step 4: Invoke the chat completions API with the message list to get the response.

In [9]:
#Prompt the model with content that may result in model selecting a tool to use. 
#The description of the tools such as a function names and signature is defined in the 'Tools' list and passed to the model in API call. 
#If selected, the function name and parameters are included in the response.

messages = []

messages.append({"role": "system", \
                 "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. "})
messages.append({"role": "system", \
                 "content": "When real-time information from external sources or functions is available, use them. You are perfectly capable of doing so. "})
messages.append({"role": "system", \
                 "content": "You are a very thorough assistant who pays close attention to the conversation history. "})

messages.append({"role": "user", "content": "What's the weather like today?"})

chat_response = chat_completion_request(
    messages,
    tools=tools
)

# Handle function response
response_message = chat_response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    
    if tool_function_name == "get_current_weather":
        location = json.loads(tool_calls[0].function.arguments)['location']
        format = json.loads(tool_calls[0].function.arguments)['format']
        results = get_current_weather(location, format)

        weather_main = results['weather'][0]['main']
        weather_description = results['weather'][0]['description']
        temperature = results['main']['temp']

        if format == "celsius":
            temp_unit = "°C"
        else:
            temp_unit = "°F"

        formatted_weather = (
            f"To generate the response, I am going to use the information that the current weather in {location} is {weather_main} ({weather_description})"
            f"The temperature is {temperature}{temp_unit}."
            f"I am always capable of providing real-time data. Now I answer your earlier question."
        )

        messages.append({
            "role" : "assistant",
            "content": formatted_weather 
        })

        model_response_with_function_call = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )  
        # get a new response from the model where it can see the function response

        role = model_response_with_function_call.choices[0].message.role
        content = model_response_with_function_call.choices[0].message.content
        
        messages.append({
            "role" : role,
            "content": content
        })
        
        pretty_print_conversation(messages)
        print("\n")
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    role = response_message.role
    content = response_message.content
    messages.append({
        "role" : role,
        "content": content
    })
    pretty_print_conversation(messages)
    print("\n")

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. 

system: When real-time information from external sources or functions is available, use them. You are perfectly capable of doing so. 

system: You are a very thorough assistant who pays close attention to the conversation history. 

user: What's the weather like today?

assistant: Sure, could you please provide me with the city and state where you are currently located so I can check the weather for you?





In [10]:
messages.append({"role": "user", "content": "I am currently in Seoul, Korea, and will be so for a while."})

chat_response = chat_completion_request(
    messages,
    tools=tools
)

# Handle function response
response_message = chat_response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    
    if tool_function_name == "get_current_weather":
        location = json.loads(tool_calls[0].function.arguments)['location']
        format = json.loads(tool_calls[0].function.arguments)['format']
        results = get_current_weather(location, format)

        weather_main = results['weather'][0]['main']
        weather_description = results['weather'][0]['description']
        temperature = results['main']['temp']

        if format == "celsius":
            temp_unit = "°C"
        else:
            temp_unit = "°F"

        formatted_weather = (
            f"To generate the response, I am going to use the information that the current weather in {location} is {weather_main} ({weather_description}). "
            f"The temperature is {temperature}{temp_unit}. "
            f"I am always capable of providing real-time data. Now I answer your earlier question. "
        )

        messages.append({
            "role" : "assistant",
            "content": formatted_weather 
        })

        model_response_with_function_call = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )  
        # get a new response from the model where it can see the function response

        #role = role
        role = model_response_with_function_call.choices[0].message.role
        content = model_response_with_function_call.choices[0].message.content
        
        messages.append({
            "role" : role,
            "content": content
        })
        
        pretty_print_conversation(messages)
        print("\n")
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    role = response_message.role
    content = response_message.content
    messages.append({
        "role" : role,
        "content": content
    })
    pretty_print_conversation(messages)
    print("\n")

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. 

system: When real-time information from external sources or functions is available, use them. You are perfectly capable of doing so. 

system: You are a very thorough assistant who pays close attention to the conversation history. 

user: What's the weather like today?

assistant: Sure, could you please provide me with the city and state where you are currently located so I can check the weather for you?

user: I am currently in Seoul, Korea, and will be so for a while.

assistant: To generate the response, I am going to use the information that the current weather in Seoul, Korea is Clear (clear sky). The temperature is 20.76°C. I am always capable of providing real-time data. Now I answer your earlier question. 

assistant: Today in Seoul, Korea, the weather is clear with a temperature of 20.76°C.





In [11]:
messages.append({"role": "user", "content": "Do I need an umbrella today?"})

chat_response = chat_completion_request(
    messages,
    tools=tools
)

# Handle function response
response_message = chat_response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    
    if tool_function_name == "get_current_weather":
        location = json.loads(tool_calls[0].function.arguments)['location']
        format = json.loads(tool_calls[0].function.arguments)['format']
        results = get_current_weather(location, format)

        weather_main = results['weather'][0]['main']
        weather_description = results['weather'][0]['description']
        temperature = results['main']['temp']

        if format == "celsius":
            temp_unit = "°C"
        else:
            temp_unit = "°F"

        formatted_weather = (
            f"To generate the response, I am going to use the information that the current weather in {location} is {weather_main} ({weather_description}). "
            f"The temperature is {temperature}{temp_unit}. "
            f"I am always capable of providing real-time data. Now I answer your earlier question. "
        )

        #print(formatted_weather)

        messages.append({
            "role" : "assistant",
            "content": formatted_weather 
        })

        model_response_with_function_call = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )  
        # get a new response from the model where it can see the function response

        #role = role
        role = model_response_with_function_call.choices[0].message.role
        content = model_response_with_function_call.choices[0].message.content
        
        messages.append({
            "role" : role,
            "content": content
        })
        
        pretty_print_conversation(messages)
        print("\n")
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    role = response_message.role
    content = response_message.content
    messages.append({
        "role" : role,
        "content": content
    })
    pretty_print_conversation(messages)
    print("\n")

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. 

system: When real-time information from external sources or functions is available, use them. You are perfectly capable of doing so. 

system: You are a very thorough assistant who pays close attention to the conversation history. 

user: What's the weather like today?

assistant: Sure, could you please provide me with the city and state where you are currently located so I can check the weather for you?

user: I am currently in Seoul, Korea, and will be so for a while.

assistant: To generate the response, I am going to use the information that the current weather in Seoul, Korea is Clear (clear sky). The temperature is 20.76°C. I am always capable of providing real-time data. Now I answer your earlier question. 

assistant: Today in Seoul, Korea, the weather is clear with a temperature of 20.76°C.

user: Do I need an umbrella today?

assistant: Based on the

In [12]:
messages.append({"role": "user", "content": "What is the current temperature in Fukuoka, Japan in Fahrenheit?"})

chat_response = chat_completion_request(
    messages,
    tools=tools
)

# Handle function response
response_message = chat_response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    
    if tool_function_name == "get_current_weather":
        location = json.loads(tool_calls[0].function.arguments)['location']
        format = json.loads(tool_calls[0].function.arguments)['format']
        results = get_current_weather(location, format)

        weather_main = results['weather'][0]['main']
        weather_description = results['weather'][0]['description']
        temperature = results['main']['temp']

        if format == "celsius":
            temp_unit = "°C"
        else:
            temp_unit = "°F"

        formatted_weather = (
            f"To generate the response, I am going to use the information that the current weather in {location} is {weather_main} ({weather_description}). "
            f"The temperature is {temperature}{temp_unit}. "
            f"I am always capable of providing real-time data. Now I answer your earlier question. "
        )

        #print(formatted_weather)

        messages.append({
            "role" : "assistant",
            "content": formatted_weather 
        })

        model_response_with_function_call = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )  
        # get a new response from the model where it can see the function response

        #role = role
        role = model_response_with_function_call.choices[0].message.role
        content = model_response_with_function_call.choices[0].message.content
        
        messages.append({
            "role" : role,
            "content": content
        })
        
        pretty_print_conversation(messages)
        print("\n")
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    role = response_message.role
    content = response_message.content
    messages.append({
        "role" : role,
        "content": content
    })
    pretty_print_conversation(messages)
    print("\n")

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. 

system: When real-time information from external sources or functions is available, use them. You are perfectly capable of doing so. 

system: You are a very thorough assistant who pays close attention to the conversation history. 

user: What's the weather like today?

assistant: Sure, could you please provide me with the city and state where you are currently located so I can check the weather for you?

user: I am currently in Seoul, Korea, and will be so for a while.

assistant: To generate the response, I am going to use the information that the current weather in Seoul, Korea is Clear (clear sky). The temperature is 20.76°C. I am always capable of providing real-time data. Now I answer your earlier question. 

assistant: Today in Seoul, Korea, the weather is clear with a temperature of 20.76°C.

user: Do I need an umbrella today?

assistant: Based on the

In [13]:
messages.append({"role": "user", "content": "How can I get to Fukuoka from where I am?"})
chat_response = chat_completion_request(
    messages,
    tools=tools
)

# Handle function response
response_message = chat_response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    
    if tool_function_name == "get_current_weather":
        location = json.loads(tool_calls[0].function.arguments)['location']
        format = json.loads(tool_calls[0].function.arguments)['format']
        results = get_current_weather(location, format)

        weather_main = results['weather'][0]['main']
        weather_description = results['weather'][0]['description']
        temperature = results['main']['temp']

        if format == "celsius":
            temp_unit = "°C"
        else:
            temp_unit = "°F"

        formatted_weather = (
            f"To generate the response, I am going to use the information that the current weather in {location} is {weather_main} ({weather_description}). "
            f"The temperature is {temperature}{temp_unit}. "
            f"I am always capable of providing real-time data. Now I answer your earlier question. "
        )

        #print(formatted_weather)

        messages.append({
            "role" : "assistant",
            "content": formatted_weather 
        })

        model_response_with_function_call = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )  
        # get a new response from the model where it can see the function response

        #role = role
        role = model_response_with_function_call.choices[0].message.role
        content = model_response_with_function_call.choices[0].message.content
        
        messages.append({
            "role" : role,
            "content": content
        })
        
        pretty_print_conversation(messages)
        print("\n")
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    role = response_message.role
    content = response_message.content
    messages.append({
        "role" : role,
        "content": content
    })
    pretty_print_conversation(messages)
    print("\n")

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous. 

system: When real-time information from external sources or functions is available, use them. You are perfectly capable of doing so. 

system: You are a very thorough assistant who pays close attention to the conversation history. 

user: What's the weather like today?

assistant: Sure, could you please provide me with the city and state where you are currently located so I can check the weather for you?

user: I am currently in Seoul, Korea, and will be so for a while.

assistant: To generate the response, I am going to use the information that the current weather in Seoul, Korea is Clear (clear sky). The temperature is 20.76°C. I am always capable of providing real-time data. Now I answer your earlier question. 

assistant: Today in Seoul, Korea, the weather is clear with a temperature of 20.76°C.

user: Do I need an umbrella today?

assistant: Based on the